In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import shutil
import pandas as pd
from source import image_id_converter as img_idc
from source import sort_img_files as sif
from source import llm_input as llm_i
from source import llm_output as llm_o
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

In [ ]:
import ollama
import json
import re
import pickle

In [ ]:
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/test_yolo_object_train')

project_path = Path.cwd()
#root_path = (project_path / 'test_LLM_prompt_experiments').resolve()
#root_path = (project_path / '..' / 'test_yolo_object_train').resolve()
#root_path = project_path /'test_llm_img_analysis'
root_path = project_path /'test_LLM_prompt_experiments'
#root_path = project_path
data_path = root_path / 'data'
tif_data_path = root_path / 'data_1'
#data_path = root_path / 'visual_genome_data_all'
jpg_data_path = root_path / 'data_jpg'
#yolo_path = root_path / 'visual_genome_yolo_all'
output_dir_not_photo = root_path / 'not_photo'
output_dir_with_person = root_path / 'with_person'
output_dir_without_person = root_path / 'without_person'

In [ ]:
os.listdir(data_path)

### Get list of potentially relevant files: 

In [ ]:
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
pkl_files = [f for f in os.listdir(data_path) if f.endswith('.pkl')]

In [ ]:
csv_files

In [ ]:
pkl_files

### Load labels data: 

In [ ]:
label_data_path = os.path.join(data_path, 'labels_mod.csv')
label_data = pd.read_csv(label_data_path)

# Reconvert image ids to integers (e.g. '234') as strings from the form they were saved in (e.g. 'id234' to ensure 
# string data type to deal with duck typing): 
img_ids = list(label_data.image_id)
label_data['image_id'] = img_idc.reconvert_image_ids(img_ids)

label_data.head()

### Set timestamp_id:

In [ ]:
#timestamp_id = '20251024_114055'
#timestamp_id = '20251025_123936'
timestamp_id = '2025-10-25 21:04'
#timestamp_id = '20251025_152957'

### Set results_tabular (results per image) file name-tag:

In [ ]:
#results_name_tag = 'results_tabular_struct_minicpm_'
#results_name_tag = 'results_multi_object_struct_minicpm_'
results_name_tag = 'results_table_prompt_exp_struct_minicpm_'
#results_name_tag = 'results_multi_object_struct_minicpm_'


### Set ml_metrics file name-tag:

In [ ]:
#ml_metrics_tag = 'ml_metrics_struct_minicpm_'
#ml_metrics_tag = 'ml_metrics_multi_object_struct_minicpm_'
ml_metrics = 'ml_metrics_prompt_exp_struct_minicpm_2025-10-25.csv'
#ml_metrics_tag = 'ml_metrics_multi_object_struct_minicpm_'

### Set times (duration of tasks) filename tag:

In [ ]:
#times_tag = 'times_struct_minicpm_'
#times_tag = 'times_multi_object_struct_minicpm_'
times_tag = 'times_df_prompt_exp_struct_minicpm_'
#times_tag = 'times_multi_object_struct_minicpm_'

### Set response raw data filename tag: 

In [ ]:
#responses_tag = 'responses_struct_minicpm_'
#responses_tag = 'responses_multi_object_struct_minicpm_'
responses_tag = 'results_prompt_exp_struct_minicpm_'
#responses_tag = 'responses_multi_object_struct_minicpm_'


### Load ml_metrics data: 

In [ ]:
# load ml_metrics:

#filename= ml_metrics_tag + timestamp_id  + '.csv'
filename = ml_metrics
ml_metrics_output_path = os.path.join(data_path, filename)

# Save csv-file: 
#ml_metrics.to_csv(ml_metrics_output_path, index=False)

# Reload saved csv table to check if saving worked:
ml_metrics_reloaded = pd.read_csv(ml_metrics_output_path)
ml_metrics_reloaded.head()

In [ ]:
ml_metrics_reloaded

### Load results data: 

In [ ]:
# load labels_results:
filename = results_name_tag + timestamp_id + '.pkl'
results_tabular_path = os.path.join(data_path, filename)
# Reload saved dictionary to check if saving worked:
with open(results_tabular_path, 'rb') as f:
   reloaded_results_tabular = pickle.load(f)

In [ ]:
print(type(reloaded_results_tabular))
print(len(reloaded_results_tabular))
print(reloaded_results_tabular.keys())


In [ ]:
timestamp_id

In [ ]:
print(type(reloaded_results_tabular[timestamp_id]))
print(len(reloaded_results_tabular[timestamp_id]))
print(reloaded_results_tabular[timestamp_id].keys())

In [ ]:
#reloaded_results_tabular['is_photo_basic_struct_minicpm']

In [ ]:
type(reloaded_results_tabular[timestamp_id]['predictions'])

In [ ]:
reloaded_results_tabular[timestamp_id]['predictions']['image_is_photograph'].head()

In [ ]:
label_data_m = label_data.copy()
for key, item in reloaded_results_tabular[timestamp_id]['predictions'].items():
    print(key)
    print(type(item))
    
    label_data_m = label_data_m.merge(item, how='inner', on='image_id')
    print(label_data_m.head())
    

In [ ]:
label_data_m.head()

In [ ]:
print(reloaded_results_tabular.keys())

In [ ]:
reloaded_results_tabular['is_photo_basic_struct_minicpm'].head()

### Load raw data (llm responses):

In [ ]:

# Define file name: 
filename = responses_tag + timestamp_id + '.pkl'

# Save dictionary with LLM responses:
llm_responses_path = os.path.join(data_path, filename)
# 
# Reload saved dictionary to check if saving worked:
with open(llm_responses_path, 'rb') as f:
   reloaded_image_descr = pickle.load(f)

# Check if original and reloaded dictionary are the same:
#print(len(image_descr))
#print(type(image_descr))
print(type(reloaded_image_descr))
print(len(reloaded_image_descr))

#print(image_descr.keys() == reloaded_image_descr.keys())

In [ ]:
reloaded_image_descr.keys()

In [ ]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'].keys()

In [ ]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id].keys()

In [ ]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id]['080']

In [ ]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id]['101']

In [ ]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id]['046']

In [ ]:
dd = {'image_is_photograph': False, 'high_alpine_environment': False, 'person': False, 'glacier': False, 'church': False, 'water_body': True, 'other_objects': ['Map of Belgium and Netherlands', 'Railway lines', 'River (Scheldt, Maas, Rhine)'], 'additional_comments': ''}
print(type(dd))
dd


In [ ]:
ml_metrics_reloaded

In [ ]:
#label_data_1 = reloaded_results_tabular['is_photo_basic_struct_minicpm'].iloc[0:12,:]
#label_data_1

In [ ]:
#label_data_3 = reloaded_results_tabular['is_photo_basic_struct_minicpm'].iloc[24:36,:]
#label_data_3

### Load time data:

In [ ]:
# load labels_results:
filename = times_tag + timestamp_id + '.pkl'
times_path = os.path.join(data_path, filename)
# Reload saved dictionary to check if saving worked:
with open(times_path, 'rb') as f:
   times_data = pickle.load(f)

In [ ]:
pd.DataFrame(times_data)

### Recalculate ml metrics: 

In [ ]:
var_name = 'is_photo'
pred_var_name = 'is_photo_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_m, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


### Get number of NaN's:

In [ ]:
num_nans = label_data_m.isna().sum().sum()
num_nans


In [ ]:
num_nulls = label_data_m.isnull().sum().sum()
num_nulls


In [ ]:
label_data_m.isna().sum()

In [ ]:
type(label_data_m.is_photo_pred[0])

### Replace nan's with 0 for confusion matrix (as NaN can be interpreted as 'not recognised as in category x'):

In [ ]:
label_data_m[label_data_m.isna().any(axis=1)]

In [ ]:
label_data_filled = label_data_m.fillna(0)
label_data_filled


In [ ]:
label_data_clean = label_data_m.dropna()
label_data_clean


### Confusion matrix 'is_photo':

#### Recalculate ml metrics: 

In [ ]:
# Recalculate ml metrics: 
var_name = 'is_photo'
pred_var_name = 'is_photo_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix: 

In [ ]:
# Plot confusion matrix: 
cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)


#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)


### Confusion matrix 'with_person':

#### Recalculate ml metrics: 

In [ ]:
# Recalculate ml metrics: 
var_name = 'with_person'
pred_var_name = 'with_person_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix:

In [ ]:

cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)

#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)

In [ ]:
false_negatives

In [ ]:
false_positives

### Confusion matrix 'with_church':

In [ ]:
# Recalculate ml metrics: 
var_name = 'with_church'
pred_var_name = 'with_church_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix:

In [ ]:

cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)


#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)

In [ ]:
false_negatives

In [ ]:
false_positives

### Confusion matrix 'in_high_alpine_environment':

In [ ]:
# Recalculate ml metrics: 
var_name = 'in_high_alpine_environment'
pred_var_name = 'in_high_alpine_environment_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix:

In [ ]:

cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)

#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)

In [ ]:
false_negatives

In [ ]:
false_positives

In [ ]:
87/89

In [ ]:
90/99